In [1]:
DATA = '../data'
IMAGES_PATH = 'images'
MASKS_PATH = 'masks'
INSTANCES_PATH = 'instance_masks'
WIDHT , HEIGHT = 224, 224
CHANNELS = 3

In [229]:
import os
import re
import imageio
import argparse
import numpy as np
import pandas as pd
import geopandas as gp
from sklearn.model_selection import StratifiedShuffleSplit


def get_data_pathes(
    datasets_path, images_path_name='images',
    masks_path_name='masks', instances_path_name='instance_masks'
    ):

    datasets = list(os.walk(datasets_path))[0][1]
    data_pathes = []
    for dataset in datasets:
        data_pathes.append((
            os.path.join(datasets_path, dataset, images_path_name),
            os.path.join(datasets_path, dataset, masks_path_name),
            os.path.join(datasets_path, dataset, instances_path_name)))
    
    return data_pathes


def get_instances(instances_path):
    return list(os.walk(instances_path))[0][1]


def image2mask(image_path, image_type):
    return imageio.imread('{}.{}'.format(image_path, image_type))


def get_data(
    images_path, masks_path, instances,
    img_type='tiff', msk_type='png'
    ):

    X = np.array([
        image2mask(os.path.join(images_path, i), img_type) for i in instances])
    y = np.array([
        image2mask(os.path.join(masks_path, i), msk_type)for i in instances])
    y = y.reshape([*y.shape, 1])
    
    return X, y


def get_area(instance_path):
    return (gp.read_file(instance_path)['geometry'].area / 100).median()

    
def get_labels(distr):
    res = np.full(distr.shape, 3)
    res[distr < np.quantile(distr, 0.75)] = 2
    res[distr < np.quantile(distr, 0.5)] = 1
    res[distr < np.quantile(distr, 0.25)] = 0
    return res


def stratify(datasets_path, test_size=0.2, random_state=42):
    images_path, masks_path, instances_path = get_data_pathes(datasets_path)[0]
    instances = list(os.walk(instances_path))[0][1]

    X, _ = get_data(images_path, masks_path, instances)
    areas = np.array([
        get_area(os.path.join(instances_path, i, i + '.geojson')) for i in instances])
    labels = get_labels(areas)

    sss = StratifiedShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state)
    
    return sss.split(X, labels)

In [224]:
def get_data_info(dataset_path):
    cols = [
        'name', 'channel', 'position',
        'image_path', 'mask_path', 'instance_path',
        'image_type', 'mask_type'
    ]
    data_info = pd.DataFrame(columns=cols)
    dataset = get_data_pathes(dataset_path)
    for subset in dataset:
        images_path, masks_path, instances_path = subset
        instances = list(os.walk(instances_path))[0][1]
        image_type = list(os.walk(images_path))[0][2][0].split('.')[-1]
        mask_type = list(os.walk(masks_path))[0][2][0].split('.')[-1]

        for i, instance in enumerate(instances):
            instance = instance.split('_')
            name = '_'.join(instance[:2])
            channel = '_'.join(instance[2:-2])
            position = '_'.join(instance[-2:])

            data_info = data_info.append(
                pd.DataFrame({
                    'name': name,
                    'channel': channel,
                    'position': position,
                    'image_path': images_path,
                    'mask_path': masks_path,
                    'instance_path': instances_path,
                    'image_type': image_type,
                    'mask_type': mask_type
                }, index=[0]),
                sort=True, ignore_index=True)
    
    return data_info


def filter_by_channel(data_info, channel_name):
    return data_info[data_info['channel'] == channel_name] \
        .sort_values(by=['position']) \
        .reset_index(drop=True)


def stratified_split(
    dataset_path, test_size=0.2, random_state=42,
    channels=['rgb', 'ndvi', 'ndvi_color', 'b2']
    ):
    
    stratified_ix = stratify(dataset_path, test_size, random_state)
    data_info = get_data_info(dataset_path)
    channels_data = [filter_by_channel(data_info, channel) for channel in channels]
    
    train_df = pd.DataFrame(columns=data_info.columns)
    test_df = pd.DataFrame(columns=data_info.columns)
    
    for i, (train_ix, test_ix) in enumerate(stratified_ix):
        for channel in data_info['channel'].unique():
            row = data_info.iloc[train_ix]
            row = row.replace(
                row['channel'].values[0],
                channel)
            train_df = train_df.append(
                row, sort=False,
                ignore_index=True)
            
            row = data_info.iloc[test_ix]
            row = row.replace(
                row['channel'].values[0],
                channel)
            test_df = test_df.append(
                row, sort=False,
                ignore_index=True)
    
    return train_df, test_df  
    

In [301]:
def build_batch_generator(
    data_info, indexes, batch_size=4,
    channels=['rgb', 'ndvi', 'ndvi_color', 'b2']
    ):
    
    if len(channels) == 0:
        raise Exception('You have to set at least 1 channel.')
        
    for i in indexes:
        images = []
        masks = []            
        for _ in range(batch_size):
            res_image = []
            res_mask = []
            for channel in channels:
                channel_data = filter_by_channel(data_info, channel)
                filename = '_'.join([
                    channel_data['name'].values[i],
                    channel,
                    channel_data['position'].values[i]])
                image_path = os.path.join(
                    channel_data['image_path'].values[i],
                    '{}.{}'.format(filename, channel_data['image_type'].values[i]))
                mask_path = os.path.join(
                    channel_data['mask_path'].values[i],
                    '{}.{}'.format(filename, channel_data['mask_type'].values[i]))
                
                image = imageio.imread(image_path)
                if image.ndim == 2:
                    image= image.reshape(*image.shape, 1)
                
                res_image.append(image)

            images.append(np.concatenate(res_image, axis=2))
            mask = imageio.imread(mask_path)
            mask = mask.reshape(*mask.shape, 1)
            masks.append(mask)
            
        masks = np.array(masks, np.float32)
        images = np.array(images, np.float32)

        yield images, masks

In [302]:
data_info = get_data_info(DATA)
train_ix, test_ix = tuple(stratify(DATA))[0]

In [304]:
gen = build_batch_generator(data_info, train_ix, channels=['rgb', 'ndvi_color'])
j = 0
a = []
for i in gen:
    a.append(i)
    print(i[0].shape)
    print(i[1].shape)

(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 7)
(4, 224, 224, 1)
(4, 224, 224, 

In [8]:
import matplotlib.pyplot as plt
plt.imshow(a[1][0][0,:,:,2])
plt.show()
plt.imshow(a[1][1][0,:,:,0])

IndexError: too many indices for array

In [305]:
train_generator = build_batch_generator(data_info, train_ix)
val_generator = build_batch_generator(data_info, test_ix)

In [306]:
import threading


def freeze_model(model, freeze_before_layer):
    if freeze_before_layer == "ALL":
        for l in model.layers:
            l.trainable = False
    else:
        freeze_before_layer_index = -1
        for i, l in enumerate(model.layers):
            if l.name == freeze_before_layer:
                freeze_before_layer_index = i
        for l in model.layers[:freeze_before_layer_index]:
            l.trainable = False

class ThreadsafeIter(object):
    def __init__(self, it):
        self.lock = threading.Lock()
        self.it = it.__iter__()

    def __iter__(self): return self

    def __next__(self):
        with self.lock:
            return next(self.it)

In [307]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_dice_loss
from segmentation_models.metrics import dice_score
import math 

BACKBONE = 'resnet50'
EPOCHS = 10
BATCH = 4
preprocess_input = get_preprocessing(BACKBONE)

model = Unet(BACKBONE, encoder_weights='imagenet')
model.compile('Adam', loss=bce_dice_loss, metrics=[dice_score])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


/home/nigi4/anaconda3/lib/python3.6/site-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [308]:
model.fit_generator(
    ThreadsafeIter(train_generator),
    steps_per_epoch=math.ceil(train_df.shape[0] / BATCH),
    epochs = 10,
    validation_data=ThreadsafeIter(val_generator),
    validation_steps=math.ceil(test_df.shape[0] / BATCH),
    max_queue_size=50,
    workers=4
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10


ValueError: Error when checking input: expected data to have shape (None, None, 3) but got array with shape (224, 224, 9)